<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/News_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import html

class News:
  def __init__(self):
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_ranking(self, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    page = str(page)
    url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date=' + date + '&page=' + page
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    div = bs_obj.find_all('div', {'class':'hotNewsList'})
    return [html.unescape(x['href']) for x in div[0].find_all('a')]

  def get_main(self, date='1'):
    date = str(datetime.datetime.now().date().strftime("%Y-%m-%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/mainnews.nhn'
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    return [x.find_all('a')[0]['href'] for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    
  def get_breaking(self):
    pass

  # 시황 전망: 401
  # 기업 종목분석: 402
  # 해외 증시: 403
  # 채권 선물: 404
  # 공시 메모: 406
  # 환율: 429
  def get_focus(self, code=403, date='1', page=1):
    date = str(datetime.datetime.now().date().strftime("%Y%m%d")) if date == '1' else date
    url = 'https://finance.naver.com/news/news_list.nhn?mode=LSS3D&section_id=101&section_id2=258&section_id3='\
     + str(code) +  '&date=' + str(date) + '&page=' + str(page)
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    temp = [html.unescape(x.find_all('a')[0]['href']) for x in bs_obj.find_all('dd', {'class':'articleSubject'})]
    # 업데이트 가능
    return [x.replace('§', '&sect') for x in temp]

  def get_article(self, url):
    url = 'https://finance.naver.com/' + url
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    title = ' '.join(bs_obj.find_all('div', {'class': 'article_info'})[0].find_all('h3')[0].get_text().split())
    context = bs_obj.find_all('div', {'class': 'articleCont'})[0]
    try:
      subheading = context.find_all('strong')[0].get_text()
    except:
      subheading = ''
    t = ' '.join(context.get_text().split())
    text = t[:t.find('@')]
    return title, subheading, text

In [2]:
news = News()

In [3]:
dic = {'Title':[], 'Subheading':[], 'Text':[]}

for i in news.get_ranking():
  title, subheading, text = news.get_article(i)
  dic['Title'].append(title)
  dic['Subheading'].append(subheading)
  dic['Text'].append(text)

In [5]:
df = pd.DataFrame.from_dict(dic)

In [9]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 46.1MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
# https://wikidocs.net/22530
# 불용어 제거 

from konlpy.tag import Okt  
okt=Okt()  
print(okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))  

class Analyser:
  def __init__(self):
    pass

  def get_tokenized(self, text):
    return okt.pos(df['Text'][0])

